In [11]:
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import (
        RunnablePassthrough)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from dotenv import load_dotenv
import numpy as np

load_dotenv()

True

In [12]:
sample_documents = [
    Document(
        page_content=""",
                Artificial Intelligence (AI) is the simulation of human intelligence in machines.",
                These systems are designed to think like humans and mimic their actions.,
                AI can be categorized into narrow AI and general AI.""",
        metadata={"source": "AI Introduction", "page": 1, "topic": "AI"}
    ),
    Document(
        page_content="""
        Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.
        """,
        metadata={"source": "ML Basics", "page": 1, "topic": "ML"}
    ),
    Document(
        page_content="""
        Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning has revolutionized computer vision, NLP, and speech recognition.
        """,
        metadata={"source": "Deep Learning", "page": 1, "topic": "DL"}
    ),
    Document(
        page_content="""
        Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learning models.
        Applications include chatbots, translation, sentiment analysis, and text summarization.
        """,
        metadata={"source": "NLP Overview", "page": 1, "topic": "NLP"}
    )
]

print(sample_documents)

[Document(metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}, page_content=',\n                Artificial Intelligence (AI) is the simulation of human intelligence in machines.",\n                These systems are designed to think like humans and mimic their actions.,\n                AI can be categorized into narrow AI and general AI.'), Document(metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='\n        Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.\n        '), Document(metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='\n        Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep l

In [13]:
## text splitting
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=[" "]
)
## split the documents into chunks
chunks = text_splitter.split_documents(sample_documents)
print(chunks[0])
print(chunks[1])

page_content=',
                Artificial Intelligence (AI) is the simulation of human intelligence in machines.",
                These systems are designed to think like humans and mimic their actions.,
                AI can be categorized into narrow AI and general AI.' metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}
page_content='Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.' metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}


In [14]:
print(f"Created {len(chunks)} chunks from {len(sample_documents)} documents")
print("Example chunk:")
print(f"Content: {chunks[0].page_content}")
print(f"Metadata: {chunks[0].metadata}")

Created 4 chunks from 4 documents
Example chunk:
Content: ,
                Artificial Intelligence (AI) is the simulation of human intelligence in machines.",
                These systems are designed to think like humans and mimic their actions.,
                AI can be categorized into narrow AI and general AI.
Metadata: {'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}


In [15]:
### load the embedding models\n",
import os
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [16]:
# Initialize OpenAI embeddings with the latest model
embeddings=OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1536
)

## Example: create a embedding for a single text
sample_text="What is machine learning"
sample_embedding=embeddings.embed_query(sample_text)
sample_embedding

[-0.0059221177361905575,
 -0.005889697000384331,
 0.0005751235294155777,
 -0.033544257283210754,
 0.021192103624343872,
 0.02213229238986969,
 -0.0007868689717724919,
 0.00929923728108406,
 -0.022564563900232315,
 0.037823744118213654,
 0.015183531679213047,
 -0.03676467761397362,
 -0.03397652879357338,
 0.0016048074467107654,
 0.026735983788967133,
 0.016015654429793358,
 0.0008530605118721724,
 -0.030864175409078598,
 0.023385880514979362,
 0.004903578199446201,
 3.242035018047318e-05,
 0.032550033181905746,
 0.042276136577129364,
 -0.029307996854186058,
 0.015961619094014168,
 -0.02092193439602852,
 0.0266279149800539,
 0.01239538099616766,
 0.00744047062471509,
 -0.006019378546625376,
 -0.009423515759408474,
 -0.029027020558714867,
 -0.03127483278512955,
 0.02755729854106903,
 0.04173579812049866,
 0.00043159592314623296,
 -0.0080564571544528,
 -0.010633875615894794,
 -0.055503640323877335,
 0.0025855230633169413,
 -0.05632495880126953,
 -0.016264209523797035,
 0.03028060868382454,

In [17]:
texts=["AI","Machine learning","Deep Learning","Neural Network"]
batch_embeddings=embeddings.embed_documents(texts)
print(batch_embeddings[0])

[-0.008208601735532284, -0.024612585082650185, 0.002946041990071535, 0.025167757645249367, 0.006533174309879541, -0.02826085314154625, -0.0050229765474796295, 0.020977536216378212, -0.036879222840070724, 0.012868072837591171, -0.0030385705176740885, -0.020144781097769737, 0.000270769844064489, -0.032781533896923065, 0.006414209026843309, -0.02529994025826454, -0.031010271981358528, -0.05440676957368851, 0.03280796855688095, -0.01842639409005642, 0.01658904179930687, 0.04832632467150688, -0.024876952171325684, 0.014341920614242554, 0.0293976329267025, 0.004044818226248026, 0.009246242232620716, 0.0133439339697361, 0.0025065315421670675, -0.02259017713367939, 0.03214704990386963, -0.028022922575473785, 0.005317085422575474, -0.038227494806051254, -0.0167080070823431, 0.014355138875544071, -0.03859760984778404, -0.01037641242146492, -0.01056146901100874, -0.01924593187868595, 0.03206774219870567, 0.014632724225521088, -0.02155914530158043, 0.016126397997140884, -0.011843650601804256, 0.00

In [18]:
### Compare Embedding using cosine similarity
def compare_embeddings(text1:str,text2:str):
    """Compare semantic simialrity of 2 texts usign embeddings"""

    emb1=np.array(embeddings.embed_query(text1))
    emb2=np.array(embeddings.embed_query(text2))
    emb1, emb2
    ## Calculate the simialrity score

    similarity=np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    return similarity

In [19]:
# Test semantic similarity
print("Semantic Similarity Examples:")
print(f"'AI' vs 'Artificial Intelligence': {compare_embeddings('AI', 'Artificial Intelligence'):.3f}")

Semantic Similarity Examples:
'AI' vs 'Artificial Intelligence': 0.563


In [20]:
print(f"'AI' vs 'Pizza': {compare_embeddings('AI', 'Pizza'):.3f}")

'AI' vs 'Pizza': 0.254


In [21]:
print(f"'Machine Learning' vs 'ML': {compare_embeddings('Machine Learning', 'ML'):.3f}")

'Machine Learning' vs 'ML': 0.461


In [22]:
### Create FAISS Vector Store
vectorstore=FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)
print(f"Vector store created with {vectorstore.index.ntotal} vectors")

Vector store created with 4 vectors


In [23]:
## Save vector store for later use
vectorstore.save_local("faiss_index")
print("Vector store saved to 'faiss_index' directory")

Vector store saved to 'faiss_index' directory


In [24]:
## load vector store
loaded_vectorstore=FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)

print(f"Loaded vector store contains {loaded_vectorstore.index.ntotal} vectors")

Loaded vector store contains 4 vectors


In [25]:
## Similarity Search
query="What is deep learning"
results=vectorstore.similarity_search(query,k=3)
print(results)

[Document(id='952c5824-2c97-4c2c-8b98-6be002ef1e4c', metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolutionized computer vision, NLP, and speech recognition.'), Document(id='e0da11ad-7f89-4b32-9d13-352fb4036d39', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.'), Document(id='14900207-ba90-4b27-b926-8825f3996159', metadata={'source': 'NLP Overview', 'page': 1, 'topic': 'NLP'}, page_content='Natural Language Processing (NLP) is a branch of AI that helps computers understand human lang

In [26]:
print(f"Query: {query}")
print("Top 3 similar chunks:")
for i, doc in enumerate(results):
    print(f"\n{i+1}. Source: {doc.metadata['source']}")
    print(f"   Content: {doc.page_content[:200]}...")

Query: What is deep learning
Top 3 similar chunks:

1. Source: Deep Learning
   Content: Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning ...

2. Source: ML Basics
   Content: Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised...

3. Source: NLP Overview
   Content: Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learning models.
      ...


In [27]:
### Similarity Search with score
results_with_scores=vectorstore.similarity_search_with_score(query,k=3)

print("Similarity search with scores:")
for doc, score in results_with_scores:
    print(f"Score: {score:.3f}")
    print(f"Source: {doc.metadata['source']}")
    print(f"Content preview: {doc.page_content[:100]}...")

Similarity search with scores:
Score: 0.556
Source: Deep Learning
Content preview: Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses m...
Score: 1.208
Source: ML Basics
Content preview: Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being...
Score: 1.274
Source: NLP Overview
Content preview: Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
...


In [28]:
### Search with metadata filtering
filter_dict={"topic":"ML"}
filtered_results=vectorstore.similarity_search(
    query,
    k=3,
    filter=filter_dict
)
print(filtered_results)

[Document(id='e0da11ad-7f89-4b32-9d13-352fb4036d39', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.')]


In [29]:
len(filtered_results)

1

In [30]:
### Build RAG Chain With LCEL 
from langchain.chat_models import init_chat_model

llm=init_chat_model("openai:gpt-3.5-turbo")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000023035F15FD0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000023035F15490>, root_client=<openai.OpenAI object at 0x000002303670B4A0>, root_async_client=<openai.AsyncOpenAI object at 0x0000023035F15D00>, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [31]:
llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CLq5Kv4Bpr7MQa3UEA7jpQLt0ai6N', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--44909ead-e96a-4603-8164-e104e0237859-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [33]:
# 1. Simple RAG Chain with LCEL
simple_prompt = ChatPromptTemplate.from_template("""Answer the question based only on the following context:
    Context: {context}
    Question: {question}
    Answer:""")

In [34]:
## Basic retriever
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)

In [35]:
from typing import List

def format_docs(docs: List[Document]) -> str:
    """Format documents for insertion into prompt"""
    formatted = []
    for i, doc in enumerate(docs):
        source = doc.metadata.get('source', 'Unknown')
        formatted.append(f"Document {i+1} (Source: {source}):\n{doc.page_content}")
    return "\n\n".join(formatted)

In [36]:
simple_rag_chain=(
       {"context":retriever | format_docs,"question":RunnablePassthrough()}
        | simple_prompt
        | llm
        | StrOutputParser()
    )

In [37]:
simple_rag_chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023036726210>, search_kwargs={'k': 3})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n    Context: {context}\n    Question: {question}\n    Answer:'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000023035F15FD0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000023035F15490>, root_client=<openai.OpenAI object at 0x000002303670B4A0>, root_async_client=<openai.AsyncOpenAI object at 0x0000023035F15D00>,

In [38]:
### Conversational RAg Chain
conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the provided context to answer questions."),
    ("placeholder", "{chat_history}"),
    ("human", "Context: {context} Question: {input}"),
])

In [47]:
def create_conversational_rag():
    """Create a conversational RAG chain with memory"""
    return (
            RunnablePassthrough.assign(
            context=lambda x: format_docs(retriever.invoke(x["input"]))
        )
        | conversational_prompt
        | llm
        | StrOutputParser()
    )
    
conversational_rag = create_conversational_rag()

In [48]:
conversational_rag

RunnableAssign(mapper={
  context: RunnableLambda(lambda x: format_docs(retriever.invoke(x['input'])))
})
| ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag=

In [49]:
### streaming RAG chain
streaming_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | simple_prompt
    | llm
)
print("Modern RAG chains created successfully!")
print("Available chains:")
print("- simple_rag_chain: Basic Q&A")
print("- conversational_rag: Maintains conversation history")
print("- streaming_rag_chain: Supports token streaming")

Modern RAG chains created successfully!
Available chains:
- simple_rag_chain: Basic Q&A
- conversational_rag: Maintains conversation history
- streaming_rag_chain: Supports token streaming


In [50]:
# Test function for different chain types
def test_rag_chains(question: str):
    """Test all RAG chain variants"""
    print(f"Question: {question}")
    print("=" * 80)

    # 1. Simple RAG
    print("\n1. Simple RAG Chain:")
    answer = simple_rag_chain.invoke(question)
    print(f"Answer: {answer}")

    print("\n2. Streaming RAG:")
    print("Answer: ", end="", flush=True)
    for chunk in streaming_rag_chain.stream(question):
        print(chunk.content, end="", flush=True)
    print()

In [51]:
test_rag_chains("What is the difference between AI and machine learning")

Question: What is the difference between AI and machine learning

1. Simple RAG Chain:
Answer: AI is the simulation of human intelligence in machines, while machine learning is a subset of AI that enables systems to learn from data. Machine learning algorithms find patterns in data instead of being explicitly programmed.

2. Streaming RAG:
Answer: AI is the simulation of human intelligence in machines, while machine learning is a subset of AI that enables systems to learn from data by finding patterns in data.


In [52]:
# Test with multiple questions
test_questions = [
    "What is the difference between AI and Machine Learning?",
    "Explain deep learning in simple terms",
    "How does NLP work?"
]

for question in test_questions:
    print("\n" + "=" * 80 + "\n")
    test_rag_chains(question)



Question: What is the difference between AI and Machine Learning?

1. Simple RAG Chain:
Answer: The main difference between AI and Machine Learning is that AI is the broader concept of simulating human intelligence in machines, while Machine Learning is a subset of AI that specifically focuses on systems learning from data and finding patterns without being explicitly programmed.

2. Streaming RAG:
Answer: AI is the simulation of human intelligence in machines, while Machine Learning is a subset of AI that enables systems to learn from data. In AI, systems are designed to think like humans and mimic their actions, while in Machine Learning algorithms find patterns in data instead of being explicitly programmed.


Question: Explain deep learning in simple terms

1. Simple RAG Chain:
Answer: Deep learning is a type of machine learning that uses neural networks with multiple layers to process and understand complex data, allowing computers to extract important features and patterns from

In [54]:
## Conversational example
print("Conversational RAG Example:")
chat_history = []
# First question
q1 = "What is machine learning?"
a1 = conversational_rag.invoke({
    "input": q1,
    "chat_history": chat_history
})
print(f"Q1: {q1}")
print(f"A1: {a1}")

Conversational RAG Example:
Q1: What is machine learning?
A1: Machine Learning is a subset of AI that allows systems to learn from data and discover patterns without being explicitly programmed. It includes algorithms like supervised, unsupervised, and reinforcement learning.


In [55]:
# Follow-up question
q2 = "How is it different from traditional programming?"
a2 = conversational_rag.invoke({
    "input": q2,
    "chat_history": chat_history,
    })
print(f"Q2: {q2}")
print(f"A2: {a2}")

Q2: How is it different from traditional programming?
A2: Traditional programming involves explicitly writing instructions to tell a computer what to do in a specific situation, while artificial intelligence, including machine learning and deep learning, enables systems to learn and make decisions based on data without being explicitly programmed for each scenario. AI systems can adapt and improve their performance over time by finding patterns in data and adjusting their behavior accordingly, which is a fundamental difference from traditional programming methods.
